The goal is to implement a car control using Udacity's simulator from Term 2.

In [1]:
from simple_websocket_server import WebSocketServer, WebSocket

In [8]:
class SimpleClient(WebSocket):
    def connected(self):
        print(self.address, 'connected')
        
    def handle(self):
        print("Handle data: " + self.data)
        response = '42["manual",{}]'
        self.send_message(response)
        

In [9]:
port = 4567
server = WebSocketServer('', port, SimpleClient)
try:
    server.serve_forever()
except KeyboardInterrupt: 
    print("Stopping execution...")
    server.close()
        

('::ffff:127.0.0.1', 54961, 0, 0) connected
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001594197}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001603373}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001603373}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.371

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001597634}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001600793}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001602029}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.7432

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001598002}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001597642}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001600726}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.7432

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001601077}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001601077}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001593783}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.7432

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001590202}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001596903}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.000159903}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.74322

...

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.00015945}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.00015945}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.00015945}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x"

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001601939}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001601939}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001594367}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.7432

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001600771}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001600771}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001600771}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.7432

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001602243}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001602243}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001594935}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.7432

Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001589991}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001589991}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.743225,"x":-76.58183,"y":84.2461,"steering_angle":0,"throttle":0,"speed":0.0001589991}]
Handle data: 42["telemetry",{"ptsx":[-61.09,-78.29172,-93.05002,-107.7717,-123.3917,-134.97],"ptsy":[92.88499,78.73102,65.34102,50.57938,33.37102,18.404],"psi_unity":4.110756,"psi":3.7432